In [ ]:
from pipeline import (univariate_data_analysis, load_data, split_data, 
split_univariate_sequence, time_series_forecasting, index_date)
import pandas as pd


In [ ]:
df_filtered, df_info_filtered = load_data()

In [ ]:
def index_date(ts):
    starting_date = df_info_filtered[df_info_filtered['M4id'] == ts]['StartingDate'].iloc[0]
    date_range = pd.date_range(start=starting_date, periods=len(df_filtered[ts]), freq='H')
    df_filtered[ts].index = date_range
    return df_filtered[ts].dropna()

ts = index_date('H38')

In [ ]:
univariate_data_analysis(ts)

In [ ]:
train_data, val_data, test_data = split_data(ts)
best_model = time_series_forecasting(train_data, val_data, test_data)